## Урок 6. Задача look-alike##

Домашнее задание
1.взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)
2.сделать feature engineering
3.обучить любой классификатор (какой вам нравится)
4.далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть
5.применить random negative sampling для построения классификатора в новых условиях
6.сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)

In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('C:\\Users\\Mell\\Desktop\\materials_6\\adult.data', sep=',', index_col=False,
    names=['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship',
           'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income'])

In [4]:
df.head(1)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K


In [5]:
df['income'] = df['income'].map({' <=50K': 0, ' >50K': 1})

In [6]:
pd.DataFrame([df.isna().sum(), df.isnull().sum()], index={'na', 'null'})

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
na,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
from sklearn.model_selection import train_test_split
X = pd.get_dummies(df.iloc[:, :-1])
y = df['income']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=33)

In [10]:
import xgboost as xgb

model = xgb.XGBClassifier()
model.fit(X_train, y_train)
y_predict = model.predict(X_test)

In [11]:
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    print("precision: %.2f%%" % (prc * 100.0)) 

    
evaluate_results(y_test, y_predict)

Classification results:
f1: 72.07%
roc: 80.17%
recall: 65.90%
precision: 79.50%


In [12]:
mod_data = pd.get_dummies(df.copy())
mod_data['income'].value_counts()

0    24720
1     7841
Name: income, dtype: int64

In [13]:
# get the indices of the positives samples
pos_ind = np.where(mod_data.loc[:, 'income'].values == 1)[0]

# shuffle them
np.random.shuffle(pos_ind)

# leave just 25% of the positives marked
pos_sample_len = int(np.ceil(0.25 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 1961/7841 as positives and unlabeling the rest


Создаем столбец для новой целевой переменной, где у нас два класса - P (1) и U (-1)

In [14]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample, 'class_test'] = 1
print('target variable:\n', mod_data.iloc[:, -1].value_counts())

target variable:
 -1    30600
 1     1961
Name: class_test, dtype: int64


In [15]:
x_data = mod_data.drop(['income', 'class_test'], axis=1).values  # just the X
y_labeled = mod_data.loc[:, 'class_test'].values  # new class (just the P & U)
y_positive = mod_data.loc[:, 'income'].values  # original class

**Random negative sampling**

In [16]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test'] == -1][:len(mod_data[mod_data['class_test'] == 1])]
sample_test = mod_data[mod_data['class_test'] == -1][len(mod_data[mod_data['class_test'] == 1]):]
pos_sample = mod_data[mod_data['class_test'] == 1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(1961, 110) (1961, 110)


In [17]:
model_rns = xgb.XGBClassifier()
model.fit(sample_train.drop(['income', 'class_test'], axis=1).values,
          sample_train.loc[:, 'income'].values)
y_predict_rns = model.predict(sample_test.drop(['income', 'class_test'], axis=1).values)

evaluate_results(sample_test.loc[:, 'income'].values, y_predict_rns)

Classification results:
f1: 60.81%
roc: 82.34%
recall: 89.43%
precision: 46.06%


In [18]:
def evaluate_results1(y_test, y_predict):
    from sklearn.metrics import recall_score, precision_score, f1_score, roc_auc_score
    f1 = f1_score(y_test, y_predict)
    roc = roc_auc_score(y_test, y_predict)
    precision = precision_score(y_test, y_predict, average='binary')
    recall = recall_score(y_test, y_predict, average='binary')
    return f1, precision, recall, roc

In [19]:
pd.DataFrame([
    evaluate_results1(y_test, y_predict),
    evaluate_results1(sample_test.loc[:, 'income'].values, y_predict_rns)], 
    columns={'f1', 'precision', 'recall', 'roc'}, 
    index={'Simple xgboost', 'Random negative sampling'}).round(3)

,precision,recall,roc,f1
Random negative sampling,0.721,0.795,0.659,0.802
Simple xgboost,0.608,0.461,0.894,0.823
